# ERK-KTR Full FOV Stimulation Pipeline

## Experimental Settings

## System Init

### Load pymmcore and required python libraries

In [ ]:
import os
import time

os.environ["QT_LOGGING_RULES"] = (
    "*.debug=false; *.warning=false"  # Fix to suppress PyQT warnings from napari-micromanager when running in a Jupyter notebook
)
os.environ["MICROMANAGER_PATH"] = "C:\\Program Files\\Micro-Manager-2.0"

from rtm_pymmcore.data_structures import Fov, Channel, StimTreatment
from rtm_pymmcore.utils import create_folders
from pprint import pprint
import pandas as pd
import numpy as np
import dataclasses
import random
import napari
import pymmcore_plus
from napari_micromanager import MainWindow

from useq._mda_event import SLMImage

mmc = pymmcore_plus.CMMCorePlus()

### Device Specific Init for Niesen Microscope

In [2]:
mmc.loadSystemConfiguration(
    "C:\\Program Files\Micro-Manager-2.0\\MMConfig_demo.cfg"
)
mmc.setConfig(groupName="System", configName="Startup")
mmc.setChannelGroup(channelGroup="Channel")

## GUI - Napari Micromanager

### Load GUI

In [3]:
### Base GUI ###
viewer = napari.Viewer()
mm_wdg = MainWindow(viewer)
viewer.window.add_dock_widget(mm_wdg)

In [4]:
### Add MDA widget for FOV selection ###
from pymmcore_widgets.mda import MDAWidget

mdawidget = MDAWidget(mmcore=mmc)
viewer.window.add_dock_widget(mdawidget)

### Functions to break and re-connect link with GUI if manually broken

The following functions can be used to manually interrupt to connection between the GUI and the running rtm-pymmcore script. However, normally you don't need to execute them. 

In [ ]:
### Break connection
# mm_wdg._core_link.cleanup()

In [ ]:
### Manually reconnect pymmcore with napari-micromanager
from napari_micromanager._core_link import CoreViewerLink

mm_wdg._core_link = CoreViewerLink(viewer, mmc)

## Create a DF with all planned acquisitions and stimulations

### Settings for Experiment

In [ ]:
df_acquire = pd.DataFrame(
    columns=[
        "fov",
        "timestep",
        "time",
        "time_experiment",
        "treatment",
        "acquired",
        "stim",
        "channels",
        "channel_stim",
    ]
)

base_path = (
    "C:\\Users\\Alex\\Ausbildung\\PhD_temp\\test_exp"
)
experiment_name = "exp_test"
path = os.path.join(base_path, experiment_name)

N_FRAMES = 2
FIRST_FRAME_STIMULATION = 10

SLEEP_BEFORE_EXPERIMENT_START = 0
USE_AUTOFOCUS_EVENT = False

create_folders(
    path, ["stim", "raw", "labels", "stim_mask", "tracks", "labels_rings", "particles"]
)

time_between_frames = 2  # time in seconds between frames
time_per_fov = 1  # time in seconds per fov

timesteps = range(N_FRAMES)

channels = []
channels.append(Channel(name="DAPI", exposure=150, group="Channel", power=2, device_name="LED", property_name="State"))
channels.append(Channel(name="Cy5", exposure=150)

# cell_lines = ["optoFGFR_high"] * 24 + ["optoFGFR"] * 24
condition = ["FGFR_high"]
# if defining individual fovs, else these values a re ignored:
n_fovs_per_cell_line = 36  ## change this variable to the amount of fovs that you have per cell line. If only one cell line is set, this value will
# automatically set to total amount of fovs. If you are working will wellplate, this value will be ignored, as each columns
# will be an entry in the cell lines list.

n_fovs_per_well = None  ## change this variable to the amount of fovs that you have per well. Set to None if you are not working with wellplate.
# If you are working with the autogenetreated wellplate, this value will be ignored, as each columns will be an entry in the cell lines list.

# stim_timesteps = [list(range(10,N_FRAMES,1)), list(range(10,N_FRAMES,2)), list(range(10,N_FRAMES,10)), list(range(10,N_FRAMES,20)), [10], list(range(10,N_FRAMES,5))]  # list of timesteps for stimulation, if e.g. double stimulation in frame 0 and 1 is needed write [[0,1]]
# stim_timesteps = [list(range(FIRST_FRAME_STIMULATION,N_FRAMES,2))]  # list of timesteps for stimulation, if e.g. double stimulation in frame 0 and 1 is needed write [[0,1]]

# stim_timesteps = [[10], list(range(10, 60, 2)), list(range(10, 60, 1)), list(range(10,60,10)), list(range(10, 60, 20)), list(range(10, 60, 5))]  # list of timesteps for stimulation, if e.g. double stimulation in frame 0 and 1 is needed write [[0,1]]


stim_exposures = [(60)]
stim_timesteps = [(10, 20), (30, 40)]


stim_treatments = [
    StimTreatment(
        stim_channel_name="FITC",
        stim_channel_group="Channels",
        stim_timestep = stim_timestep,
        stim_exposure=stim_exposure,
        stim_power=3,
        stim_channel_device_name="LED",
        stim_channel_power_property_name="State",
    )

    for stim_exposure in stim_exposures
    for stim_timestep in stim_timesteps
]
pprint(stim_treatments)
data_mda_fovs = None

Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\stim already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\raw already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\labels already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\stim_mask already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\tracks already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\labels_rings already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\particles already exists
[StimTreatment(stim_channel_name='FITC',
               stim_channel_group='Channels',
               stim_timestep=(10, 20),
               stim_exposure=60,
               stim_power=3,
               stim_channel_device_name='LED',
               stim_channel_power_property_name='State'),
 StimTreatment(stim_channel_name='FITC',
               stim_channel_group='Channels',
               stim_times

### Map Experiment to FOVs

#### If FOVs already saved - Reload them from file

In [4]:
import json

file = os.path.join(path, "fovs.json")
with open(file, "r") as f:
    data_mda_fovs = json.load(f)

Only select one of the following two code blocks. If you autogenerated FOVs using the wellplate option of the MDA widget, then use the first code block, else the second. 

#### FOVs were selected using MDA widget

### Use FOVs to generate dataframe for acquisition

In [35]:
n_fovs_simultaneously = time_between_frames // time_per_fov
start_time = 0
if data_mda_fovs is None: 
    assert False, "No fovs selected. Please select fovs in the MDA widget and save them to a json file."
dfs = []
fovs = []
for fov_index, fov in enumerate(data_mda_fovs):
    fov_object = Fov(fov_index)
    fovs.append(fov_object)
    fov_group = fov_index // n_fovs_simultaneously
    start_time = fov_group * time_between_frames * len(timesteps)
    if len(condition) == 1:
        condition_fov = condition[0]
    else:
        condition = condition[fov_index // n_fovs_per_cell_line]
    for timestep in timesteps:
        row = {
            "fov_object": fov_object,
            "fov": fov_index,
            "fov_x": fov.get("x"),
            "fov_y": fov.get("y"),
            "fov_z": fov.get("z"),
            "fov_name": str(fov_index) if fov["name"] is None else fov.name, 
            "timestep": timestep,
            "time": start_time + timestep * time_between_frames,
            "cell_line": condition_fov,
            "channels": tuple(dataclasses.asdict(channel) for channel in channels),   
            "fname": f"{str(fov_index).zfill(3)}_{str(timestep).zfill(5)}",
        }
        dfs.append(row)

df_acquire = pd.DataFrame(dfs)

print(f"Total Experiment Time: {df_acquire['time'].max()/3600}h")


n_fovs = len(data_mda_fovs)
n_stim_treatments = len(stim_treatments)
if n_stim_treatments > 0: 
    n_fovs_per_stim_condition = n_fovs // n_stim_treatments // len(np.unique(condition))
    stim_treatment_tot = []
    random.shuffle(stim_treatments)
    if n_fovs_per_well is not None:
        for stim_treat in stim_treatments:
            stim_treatment_tot.extend([stim_treat] * n_fovs_per_well)

    else:
        for fov_index in range(0, n_fovs_per_stim_condition):
            stim_treatment_tot.extend(stim_treatments)
        random.shuffle(stim_treatment_tot)

        if n_fovs % n_stim_treatments != 0:
            print(
                f"Warning: Not equal number of fovs per stim condition. {n_fovs % n_stim_treatments} fovs will have repeated treatment"
            )
            stim_treatment_tot.extend(stim_treatments[: n_fovs % n_stim_treatments])
    print(f"Doing {n_fovs_per_stim_condition} experiment per stim condition")

    if len(condition) == 1:
        n_fovs_per_cell_line = n_fovs
    else:
        stim_treatment_tot = stim_treatment_tot * len(np.unique(condition))

    df_acquire = pd.merge(df_acquire, pd.DataFrame(stim_treatment_tot), left_on="fov", right_index=True)
    
    # Add stim column that checks if current timestep is in the stim_timestep tuple
    df_acquire["stim"] = df_acquire.apply(
        lambda row: row["timestep"] in row["stim_timestep"] 
        if isinstance(row["stim_timestep"], tuple) and row["stim_exposure"] > 0 else False, 
        axis=1
    )

df_acquire = df_acquire.dropna(axis=1, how='all')
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", True)
df_acquire = df_acquire.sort_values(by=["time", "fov"])
df_acquire

Total Experiment Time: 0.0005555555555555556h
Doing 1 experiment per stim condition


,fov_object,fov,fov_x,fov_y,fov_z,fov_name,timestep,time,cell_line,channels,fname,stim_channel_name,stim_channel_group,stim_timestep,stim_exposure,stim_power,stim_channel_device_name,stim_channel_power_property_name,stim
0,"Fov(index=0, light_mask=None, stim_mask_queue=...",0,0.00,0.0,0.0,0,0,0,FGFR_high,"({'name': 'DAPI', 'exposure': 150, 'group': 'C...",000_00000,FITC,Channels,"(10, 20)",60,3,LED,State,False
2,"Fov(index=1, light_mask=None, stim_mask_queue=...",1,20.01,0.0,0.0,1,0,0,FGFR_high,"({'name': 'DAPI', 'exposure': 150, 'group': 'C...",001_00000,FITC,Channels,"(30, 40)",60,3,LED,State,False
1,"Fov(index=0, light_mask=None, stim_mask_queue=...",0,0.00,0.0,0.0,0,1,2,FGFR_high,"({'name': 'DAPI', 'exposure': 150, 'group': 'C...",000_00001,FITC,Channels,"(10, 20)",60,3,LED,State,False
3,"Fov(index=1, light_mask=None, stim_mask_queue=...",1,20.01,0.0,0.0,1,1,2,FGFR_high,"({'name': 'DAPI', 'exposure': 150, 'group': 'C...",001_00001,FITC,Channels,"(30, 40)",60,3,LED,State,False


In [ ]:
c

## Run experiment

In [18]:
%load_ext autoreload
%autoreload 2


# for _ in range(0, SLEEP_BEFORE_EXPERIMENT_START * 3600):
#     time.sleep(1)
from rtm_pymmcore.add_frame import ImageProcessingPipeline
from rtm_pymmcore.segmentation.base_segmentator import DummySegmentator
from rtm_pymmcore.stimulation.base_stim import StimWholeFOV
from rtm_pymmcore.controller import ControllerSimulated, Analyzer
from rtm_pymmcore.tracking.base_tracker import TrackerNoTracking
from rtm_pymmcore.dmd import DMD
from queue import Queue

try:
    mm_wdg._core_link.cleanup()
except:
    pass

segmentator = DummySegmentator()
stimulator = StimWholeFOV()
tracker = TrackerNoTracking()




pipeline = ImageProcessingPipeline(segmentator, stimulator, tracker, storage_path=path)
analyzer = Analyzer(pipeline)
queue = Queue()
controller = ControllerSimulated(analyzer, mmc, queue, project_path="C:\\Users\\Alex\\Ausbildung\\PhD_temp\\test_exp\\old_data")
controller.run(df_acquire)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[03.04.2025 23:12:15] INFO     MDA Started: GeneratorMDASequence()                                   ]8;id=133474;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=924964;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py#376\376]8;;\

                      INFO     index={'t': 0, 'c': 0, 'p': 0} channel=Channel(config='DAPI')         ]8;id=965480;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=117683;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py#337\337]8;;\
                               exposure=150.0 min_start_time=0.0 x_pos=0.0 y_pos=0.0 z_pos=0.0                     
                               properties=[PropertyTuple(device_name='LED', property_name='State',                 
                               property_value=2)] metadata={'fov_object': Fov(index=0,                             
                               light_mask=None, stim_mask_queue=<_queue.SimpleQueue object at                      
                               0x00000228AA665670>, tracks_queue=<_queue.SimpleQueue object at                     
                               0x00000228AA6655D0>, tracks=   0 fov_object  fov  fov_x  fov_y  fov_z               
                               fov_name  timestep  time  cell_line  \                                              
                               0  0        ...    0    0.0    0.0    0.0        0         0     0                  
                               FGFR_high                                                                           
                                                                                                                   
                                                                           channels      fname  \                  
                               0  (Channel(name='DAPI', exposure=150, group='Cha...  000_00000                     
                                                                                                                   
                                 stim_channel_name stim_channel_group stim_timestep  stim_exposure                 
                               \                                                                                   
                               0              FITC           Channels      (30, 40)             60                 
                                                                                                                   
                                  stim_power stim_channel_device_name                                              
                               stim_channel_power_property_name  \                                                 
                               0           3                      LED                                              
                               State                                                                               
                                                                                                                   
                                   stim         img_type  last_channel  \                                          
                               0  False  ImgType.IMG_RAW          True                                             
                                                                                                                   
                                                                            channel                                
                               0  Channel(name='Cy5', exposure=150, group=None, ...  , linker=None),               
                               'fov': 0, 'fov_x': 0.0, 'fov_y': 0.0, 'fov_z': 0.0, 'fov_name': '0',                
                               'timestep': 0, 'time': 0, 'cell_line': 'FGFR_high', 'channels':                     
                               (Channel(name='DAPI', exposure=150, group='Channel', power=2,                       
                               device_name='LED', property_name='State'), Channel(name='Cy5',                      
                               exposure=150, group=Non

                      INFO     index={'t': 0, 'c': 1, 'p': 0} channel=Channel(config='Cy5')          ]8;id=173309;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=742332;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py#337\337]8;;\
                               exposure=150.0 min_start_time=0.0 x_pos=0.0 y_pos=0.0 z_pos=0.0                     
                               metadata={'fov_object': Fov(index=0, light_mask=None,                               
                               stim_mask_queue=<_queue.SimpleQueue object at 0x00000228AA665670>,                  
                               tracks_queue=<_queue.SimpleQueue object at 0x00000228AA6655D0>,                     
                               tracks=   0 fov_object  fov  fov_x  fov_y  fov_z fov_name  timestep                 
                               time  cell_line  \                                                                  
                               0  0        ...    0    0.0    0.0    0.0        0         0     0                  
                               FGFR_high                                                                           
                                                                                                                   
                                                                           channels      fname  \                  
                               0  (Channel(name='DAPI', exposure=150, group='Cha...  000_00000                     
                                                                                                                   
                                 stim_channel_name stim_channel_group stim_timestep  stim_exposure                 
                               \                                                                                   
                               0              FITC           Channels      (30, 40)             60                 
                                                                                                                   
                                  stim_power stim_channel_device_name                                              
                               stim_channel_power_property_name  \                                                 
                               0           3                      LED                                              
                               State                                                                               
                                                                                                                   
                                   stim         img_type  last_channel  \                                          
                               0  False  ImgType.IMG_RAW          True                                             
                                                                                                                   
                                                                            channel                                
                               0  Channel(name='Cy5', exposure=150, group=None, ...  , linker=None),               
                               'fov': 0, 'fov_x': 0.0, 'fov_y': 0.0, 'fov_z': 0.0, 'fov_name': '0',                
                               'timestep': 0, 'time': 0, 'cell_line': 'FGFR_high', 'channels':                     
                               (Channel(name='DAPI', exposure=150, group='Channel', power=2,                       
                               device_name='LED', property_name='State'), Channel(name='Cy5',                      
                               exposure=150, group=None, power=None, device_name=None,                             
                               property_name=None)), '

gnom
   0                                         fov_object  fov  fov_x  fov_y  \
0  0  Fov(index=0, light_mask=None, stim_mask_queue=...    0    0.0    0.0   

   fov_z fov_name  timestep  time  cell_line  \
0    0.0        0         0     0  FGFR_high   

                                            channels      fname  \
0  (Channel(name='DAPI', exposure=150, group='Cha...  000_00000   

  stim_channel_name stim_channel_group stim_timestep  stim_exposure  \
0              FITC           Channels      (30, 40)             60   

   stim_power stim_channel_device_name stim_channel_power_property_name  \
0           3                      LED                            State   

    stim         img_type  last_channel  \
0  False  ImgType.IMG_RAW          True   

                                             channel  
0  Channel(name='Cy5', exposure=150, group=None, ...  


...

                      INFO     index={'t': 1, 'c': 0, 'p': 1} channel=Channel(config='DAPI')         ]8;id=154136;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=921199;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py#337\337]8;;\
                               exposure=150.0 min_start_time=2.0 x_pos=20.01 y_pos=0.0 z_pos=0.0                   
                               properties=[PropertyTuple(device_name='LED', property_name='State',                 
                               property_value=2)] metadata={'fov_object': Fov(index=1,                             
                               light_mask=None, stim_mask_queue=<_queue.SimpleQueue object at                      
                               0x00000228AA665670>, tracks_queue=<_queue.SimpleQueue object at                     
                               0x00000228AA6655D0>, tracks=   0 fov_object  fov  fov_x  fov_y  fov_z               
                               fov_name  timestep  time  cell_line  \                                              
                               0  0        ...    1  20.01    0.0    0.0        1         0     0                  
                               FGFR_high                                                                           
                                                                                                                   
                                                                           channels      fname  \                  
                               0  (Channel(name='DAPI', exposure=150, group='Cha...  001_00000                     
                                                                                                                   
                                 stim_channel_name stim_channel_group stim_timestep  stim_exposure                 
                               \                                                                                   
                               0              FITC           Channels      (10, 20)             60                 
                                                                                                                   
                                  stim_power stim_channel_device_name                                              
                               stim_channel_power_property_name  \                                                 
                               0           3                      LED                                              
                               State                                                                               
                                                                                                                   
                                   stim         img_type  last_channel  \                                          
                               0  False  ImgType.IMG_RAW          True                                             
                                                                                                                   
                                                                            channel                                
                               0  Channel(name='Cy5', exposure=150, group=None, ...  , linker=None),               
                               'fov': 1, 'fov_x': 20.01, 'fov_y': 0.0, 'fov_z': 0.0, 'fov_name':                   
                               '1', 'timestep': 1, 'time': 2, 'cell_line': 'FGFR_high', 'channels':                
                               (Channel(name='DAPI', exposure=150, group='Channel', power=2,                       
                               device_name='LED', property_name='State'), Channel(name='Cy5',                      
                               exposure=150, group=Non

                      INFO     index={'t': 1, 'c': 1, 'p': 1} channel=Channel(config='Cy5')          ]8;id=203299;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=872782;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py#337\337]8;;\
                               exposure=150.0 min_start_time=2.0 x_pos=20.01 y_pos=0.0 z_pos=0.0                   
                               metadata={'fov_object': Fov(index=1, light_mask=None,                               
                               stim_mask_queue=<_queue.SimpleQueue object at 0x00000228AA665670>,                  
                               tracks_queue=<_queue.SimpleQueue object at 0x00000228AA6655D0>,                     
                               tracks=   0 fov_object  fov  fov_x  fov_y  fov_z fov_name  timestep                 
                               time  cell_line  \                                                                  
                               0  0        ...    1  20.01    0.0    0.0        1         0     0                  
                               FGFR_high                                                                           
                                                                                                                   
                                                                           channels      fname  \                  
                               0  (Channel(name='DAPI', exposure=150, group='Cha...  001_00000                     
                                                                                                                   
                                 stim_channel_name stim_channel_group stim_timestep  stim_exposure                 
                               \                                                                                   
                               0              FITC           Channels      (10, 20)             60                 
                                                                                                                   
                                  stim_power stim_channel_device_name                                              
                               stim_channel_power_property_name  \                                                 
                               0           3                      LED                                              
                               State                                                                               
                                                                                                                   
                                   stim         img_type  last_channel  \                                          
                               0  False  ImgType.IMG_RAW          True                                             
                                                                                                                   
                                                                            channel                                
                               0  Channel(name='Cy5', exposure=150, group=None, ...  , linker=None),               
                               'fov': 1, 'fov_x': 20.01, 'fov_y': 0.0, 'fov_z': 0.0, 'fov_name':                   
                               '1', 'timestep': 1, 'time': 2, 'cell_line': 'FGFR_high', 'channels':                
                               (Channel(name='DAPI', exposure=150, group='Channel', power=2,                       
                               device_name='LED', property_name='State'), Channel(name='Cy5',                      
                               exposure=150, group=None, power=None, device_name=None,                             
                               property_name=None)), '

gnom
   0                                         fov_object  fov  fov_x  fov_y  \
0  0  Fov(index=1, light_mask=None, stim_mask_queue=...    1  20.01    0.0   

   fov_z fov_name  timestep  time  cell_line  \
0    0.0        1         1     2  FGFR_high   

                                            channels      fname  \
0  (Channel(name='DAPI', exposure=150, group='Cha...  001_00001   

  stim_channel_name stim_channel_group stim_timestep  stim_exposure  \
0              FITC           Channels      (10, 20)             60   

   stim_power stim_channel_device_name stim_channel_power_property_name  \
0           3                      LED                            State   

    stim         img_type  last_channel  \
0  False  ImgType.IMG_RAW          True   

                                             channel  
0  Channel(name='Cy5', exposure=150, group=None, ...  


                      ERROR                                                                          ]8;id=744954;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\_logger.py\_logger.py]8;;\:]8;id=8531;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\_logger.py#162\162]8;;\
                                                                                                                   
                               While emitting signal                                                               
                               'pymmcore_plus.mda.events._psygnal.MDASignaler.frameReady', an error                
                               occurred in a callback:                                                             
                                                                                                                   
                                 ArrowInvalid: ("Could not convert Channel(name='DAPI',                            
                               exposure=150, group='Channel', power=2, device_name='LED',                          
                               property_name='State') with type Channel: did not recognize Python                  
                               value type when inferring an Arrow data type", 'Conversion failed for               
                               column channels with type object')                                                  
                                 -------------------------------------------------------------------               
                               ---------------------------------------------------------------------               
                               ---------------------------------------------------------------------               
                               ---------------------------------------------------------------------               
                               -----------                                                                         
                                                                                                                   
                                 SIGNAL EMISSION:                                                                  
                                   c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore               
                               _plus\mda\_runner.py:233 in run                                                     
                                     self._run(engine, events)                                                     
                                   c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore               
                               _plus\mda\_runner.py:354 in _run                                                    
                                     self._signals.frameReady.emit(img, event, meta)  # <-- SIGNAL                 
                               WAS EMITTED HERE                                                                    
                                                                                                                   
                                 CALLBACK CHAIN:                                                                   
                                   c:\Users\Alex\Programmierung\01_git\PhD\rtm-pymmcore-1\rtm_pymmco               
                               re\controller.py:246 in _on_frame_ready                                             
                                   ... 13 more frames ...                                                          
                                   pyarrow\\error.pxi:92 in pyarrow.lib.check_status                               
                                 # <-- ERROR OCCURRED HERE                                                         
                                                                

                      INFO     MDA Finished: GeneratorMDASequence()                                  ]8;id=692469;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=345827;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py#463\463]8;;\

In [ ]:
df_new = pd.DataFrame(["1"])
len(df_new)

1